In [1]:
import os
import sys

try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    MY_DRIVE_PATH = '/content/drive/MyDrive/Technion/Diffusion-AD-Project/'
    sys.path.append(MY_DRIVE_PATH)
    os.chdir(MY_DRIVE_PATH)

except Exception as e:
    print(e)

Mounted at /content/drive


In [2]:
!pip install blobfile &> /dev/null
!pip install mpi4py &> /dev/null
!pip install pytorch-lightning &> /dev/null

In [3]:
import pytorch_lightning as pl
from IPython.display import Markdown as md
from utils.models import ModelLoader
from utils.noiser import *
from utils.denoiser import *
from utils.error_map import *
from utils.anomaly_scorer import *
from utils.data_frame_manager import DataFrameManager
from core.diffusion_ad import DiffusionAD
from extern.guided_diffusion.guided_diffusion.unet import UNetModel
from config.configuration import DIFFUSION_AD_HPARAMS, DIFFUSION_AD_REQUIRED_HPARAMS, \
    CATEGORY_TO_NOISE_TIMESTEPS, CATEGORY_TO_TYPE, CATEGORY_TYPE_OBJECT, CATEGORY_TYPE_TEXTURE

In [4]:
model: UNetModel = None

# **<u>Initialize Experiment</u>**

First, we load and construct all the needed objects for our experiment.

In [5]:
if not model:
    model, diffusion = ModelLoader().get_model('256x256_uncond')

In [6]:
noiser = TimestepUniformNoiser(diffusion)
denoiser = ModelTimestepUniformDenoiser(model, diffusion)
anomaly_map_generator = BatchFilteredSquaredError()
anomaly_scorer = MaxValueAnomalyScorer()

In [7]:
diffusion_ad = DiffusionAD(noiser,
                           denoiser,
                           anomaly_map_generator,
                           anomaly_scorer,
                           DIFFUSION_AD_HPARAMS)
diffusion_ad.args.verbosity = 1

In [8]:
trainer = {
    category: pl.Trainer.from_argparse_args(diffusion_ad.args, 
                                            default_root_dir=os.path.join(diffusion_ad.args.root_output_dir, category), 
                                            max_epochs=diffusion_ad.args.num_epochs,
                                            accelerator='gpu',
                                            devices=1) \
           for category in CATEGORY_TO_NOISE_TIMESTEPS.keys()
}

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utili

# **<u>Run Experiment</u>**

## **<u>Object Categories</u>**

First, we will run the evaluation on the different object categories.

In [9]:
OBJECT_CATEGORIES = [
    category for category, category_type in CATEGORY_TO_TYPE.items() 
        if category_type == CATEGORY_TYPE_OBJECT
]

md(f'**<u>The object categories are:</u>**\n\n{OBJECT_CATEGORIES}')

**<u>The object categories are:</u>**

['bottle', 'cable', 'capsule', 'hazelnut', 'metal_nut', 'pill', 'screw', 'toothbrush', 'transistor']

In [10]:
remaining_categories = diffusion_ad.get_remaining_categories()
overwrite = True

for category in OBJECT_CATEGORIES:
    if overwrite or category in remaining_categories:
        print('\033[4m\033[1m' + f'Evaluating category: {category}' + '\033[0m')
        diffusion_ad.args.category = category
        try:
            trainer[category].test(diffusion_ad)
        
        except FileNotFoundError as e:
            print(e, f'\nFiles missing for category {category}! Please fix your dataset folder.\n')

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Evaluating category: bottle


INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


[Errno 2] No such file or directory: '/content/drive/MyDrive/Technion/Diffusion-AD-Project/extern/mvtec/bottle/test' 
Files missing for category bottle! Please fix your dataset folder.

Evaluating category: cable
[Errno 2] No such file or directory: '/content/drive/MyDrive/Technion/Diffusion-AD-Project/extern/mvtec/cable/test' 
Files missing for category cable! Please fix your dataset folder.

Evaluating category: capsule
[Errno 2] No such file or directory: '/content/drive/MyDrive/Technion/Diffusion-AD-Project/extern/mvtec/capsule/test' 
Files missing for category capsule! Please fix your dataset folder.

Evaluating category: hazelnut


Testing: 0it [00:00, ?it/s]

...

  0%|          | 0/234 [00:00<?, ?it/s]

  0%|          | 0/232 [00:00<?, ?it/s]

Anomaly Map values:
vmin=0.0006329177948302911, vmax=0.3006066887201483
Total pixel-level auc-roc score :
0.9739718527049739
Total image-level auc-roc score :
1.0
test_epoch_end
Entered update results csv
Entered category in results data
new_dict is: {'pixel_auc': 0.9739718527049739, 'img_auc': 1.0, 'category': 'hazelnut', 'category_type': 'object'}
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
         img_auc                    1.0
        pixel_auc           0.9739718437194824
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Evaluating category: metal_nut
[Errno 2] No such file or directory: '/content/drive/MyDrive/Technion/Diffusion-AD-Project/extern/mvtec/metal_nut/test' 
Files missing for category metal_nut! Please fix your dataset folder.

Evaluating category: pill
[Errno 2] No such file or directory: '/content/drive/MyDrive/Technion/Diffusion-AD-Project/extern/mvtec/pill/test' 
Files missing for category pill! Please fix your dataset folder.

Evaluating category: screw
[Errno 2] No such file or directory: '/content/drive/MyDrive/Technion/Diffusion-AD-Project/extern/mvtec/screw/test' 
Files missing for category screw! Please fix your dataset folder.

Evaluating category: toothbrush
[Errno 2] No such file or directory: '/content/drive/MyDrive/Technion/Diffusion-AD-Project/extern/mvtec/toothbrush/test' 
Files missing for category toothbrush! Please fix your dataset folder.

Evaluating category: transistor
[Errno 2] No such file or directory: '/content/drive/MyDrive/Technion/Diffusion-AD-Project/extern/mv

## **<u>Texture Categories</u>**

Now, we will run the evaluation on the different texture categories.

In [11]:
TEXTURE_CATEGORIES = [
    category for category, category_type in CATEGORY_TO_TYPE.items() 
        if category_type == CATEGORY_TYPE_TEXTURE
]

md(f'**<u>The texture categories are:</u>**\n\n{TEXTURE_CATEGORIES}')

**<u>The texture categories are:</u>**

['carpet', 'grid', 'leather', 'tile', 'wood', 'zipper']

In [12]:
remaining_categories = diffusion_ad.get_remaining_categories()
overwrite = True

for category in TEXTURE_CATEGORIES:
    if overwrite or category in remaining_categories:
        print('\033[4m\033[1m' + f'Evaluating category: {category}' + '\033[0m')
        diffusion_ad.args.category = category
        try:
            trainer[category].test(diffusion_ad)
        
        except FileNotFoundError as e:
            print(e, f'\nFiles missing for category {category}! Please fix your dataset folder.\n')

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Evaluating category: carpet


INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


[Errno 2] No such file or directory: '/content/drive/MyDrive/Technion/Diffusion-AD-Project/extern/mvtec/carpet/test' 
Files missing for category carpet! Please fix your dataset folder.

Evaluating category: grid
[Errno 2] No such file or directory: '/content/drive/MyDrive/Technion/Diffusion-AD-Project/extern/mvtec/grid/test' 
Files missing for category grid! Please fix your dataset folder.

Evaluating category: leather
[Errno 2] No such file or directory: '/content/drive/MyDrive/Technion/Diffusion-AD-Project/extern/mvtec/leather/test' 
Files missing for category leather! Please fix your dataset folder.

Evaluating category: tile
[Errno 2] No such file or directory: '/content/drive/MyDrive/Technion/Diffusion-AD-Project/extern/mvtec/tile/test' 
Files missing for category tile! Please fix your dataset folder.

Evaluating category: wood
[Errno 2] No such file or directory: '/content/drive/MyDrive/Technion/Diffusion-AD-Project/extern/mvtec/wood/test' 
Files missing for category wood! Please